In [1]:
import os    as os

import numpy as np

import xarray as xr

import pandas as pd


In [2]:
def days_per_year(year):
    if (year % 4) == 0:
        if (year % 100) == 0:
            if (year % 400) == 0:
                days_per_year = 366
            else:
                days_per_year = 365
        else:
            days_per_year = 366
    else:
        days_per_year = 365
    return days_per_year

In [3]:
days_per_year(2100)

365

In [4]:
working_directory  = "./"

loca2_inv_file     = "./LOCA2_R1_Full_Summary_Errors.csv"

globus_root_dir    = "/data/GLOBUS/wjc"

thredds_root_dir   = "/projects/ECEP/LOCA_MACA_Ensembles/LOCA2/LOCA2_CONUS/Original_CONUS"

variables          = ["tasmax", 
                      "tasmin", 
                      "pr"]

scenarios          = ["historical", 
                      "ssp245", 
                      "ssp370", 
                      "ssp585"]

scenarios          = ["historical", 
                      "ssp245", 
                      "ssp370", 
                      "ssp585"]

hist_year_strings  = ["1950-2014"] 


futr_year_strings  = ["2015-2044",
                      "2045-2074",
                      "2075-2100"]

version_strings    = ["LOCA_16thdeg_v20220413", # tasmax
                      "LOCA_16thdeg_v20220413", # tasmin
                      "LOCA_16thdeg_v20220519"] # pr



bounds_script_command = 'defdim("bnds",2);lon_bnds=make_bounds(lon,$bnds,"lon_bnds");lat_bnds=make_bounds(lat,$bnds,"lat_bnds");time_bnds=make_bounds(time,$bnds,"time_bnds");'

In [5]:
loca2_ensembles_list = pd.read_csv(filepath_or_buffer = loca2_inv_file)
models  = loca2_ensembles_list[ "Model"].values
members = loca2_ensembles_list["Member"].values

display(loca2_ensembles_list)

,Model,Member,value,Var,Season,Stat,Rank
0,INM-CM5-0,r1i1p1f1,9.920685,All,Annual,Both,3
1,NorESM2-LM,r1i1p1f1,10.302912,All,Annual,Both,3
2,CESM2-LENS,r1i1p1f1,11.249889,All,Annual,Both,6
3,EC-Earth3,r1i1p1f1,11.582314,All,Annual,Both,4
4,IPSL-CM6A-LR,r1i1p1f1,11.812705,All,Annual,Both,6
5,EC-Earth3-Veg,r1i1p1f1,12.301219,All,Annual,Both,4
6,MPI-ESM1-2-HR,r1i1p1f1,12.387494,All,Annual,Both,4
7,MPI-ESM1-2-LR,r1i1p1f1,13.742271,All,Annual,Both,8


In [6]:
mind     = 4
mind_end = mind+1

models  =  models[mind:mind_end]
members = members[mind:mind_end]

for m in range(len(models)):
    
    model  =  models[m]
    member = members[m]

    script_name = "./" + model + "." + member  + ".csh"
    print( model + "." + member)

    f = open(file     = script_name, 
             mode     = 'w', 
             encoding = "utf-8")




    for scenario in scenarios:

        script_name0 = "./" + model + "." + member + "." + scenario + ".csh"
        log_name0    = "./" + model + "." + member + "." + scenario + ".out"
        
        print("nohup " + script_name0 + " &> " + log_name0 + " &",file=f)

    f.close()
    os.system("chmod a+x " + script_name)            



IPSL-CM6A-LR.r1i1p1f1


In [7]:
display(loca2_ensembles_list.sort_values(by = "Model"))

,Model,Member,value,Var,Season,Stat,Rank
2,CESM2-LENS,r1i1p1f1,11.249889,All,Annual,Both,6
3,EC-Earth3,r1i1p1f1,11.582314,All,Annual,Both,4
5,EC-Earth3-Veg,r1i1p1f1,12.301219,All,Annual,Both,4
0,INM-CM5-0,r1i1p1f1,9.920685,All,Annual,Both,3
4,IPSL-CM6A-LR,r1i1p1f1,11.812705,All,Annual,Both,6
6,MPI-ESM1-2-HR,r1i1p1f1,12.387494,All,Annual,Both,4
7,MPI-ESM1-2-LR,r1i1p1f1,13.742271,All,Annual,Both,8
1,NorESM2-LM,r1i1p1f1,10.302912,All,Annual,Both,3


In [8]:
for m in range(len(models)):
    
    model  =  models[m]
    member = members[m]

    
    for scenario in scenarios:
        
        script_name = "./" + model + "." + member + "." + scenario + ".csh"

        f = open(file     = script_name, 
                 mode     = 'w', 
                 encoding = "utf-8")

        
        print("# XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX", file=f)
      
        if (scenario == "historical"):
            year_strings = hist_year_strings
        else:
            year_strings = futr_year_strings
            
        thredds_dir = thredds_root_dir + "/" + scenario

        fnl_thredds_file = "LOCA2-CONUS___" + model + "." + member + "___" + scenario + ".nc"
        fnl_thredds_loc  = thredds_dir + "/" + fnl_thredds_file
        
        cat_thredds_all_yrs_file =  "LOCA2-CONUS___"  + model + "." + member + "___" + scenario + "___Y-*.nc"
        cat_thredds_all_yrs_loc  = thredds_dir + "/" + cat_thredds_all_yrs_file
                        
        for year_string in year_strings:
            print("# =========================================", file=f)
            year_bounds = year_string.split(sep="-")
            years_num = np.arange(int(year_bounds[0]),int(year_bounds[1])+1)
            years_str = str(years_num).zfill(4)
            year0 = 0
            yearN = 0


                
            for year in years_num:
                print("# -----------------------------------------", file=f)

                single_year_string = "Y-" + str(year).zfill(4)

                dpy = days_per_year(year)
                yearN = year0 + dpy - 1

                mid_wildthredds_file = "LOCA2-CONUS___*___"                + model + "." + member + "___" + scenario + "___" + single_year_string + ".nc"
                cat_thredds_file     = "LOCA2-CONUS___"                    + model + "." + member + "___" + scenario + "___" + single_year_string + ".nc"

                mid_wildthredds_loc = thredds_dir + "/" + mid_wildthredds_file
                cat_thredds_loc     = thredds_dir + "/" + cat_thredds_file


                
                print("# .........................................", file=f)

                for v in range(len(variables)):
                    print("# . . . . . . . . . . . . . . . . . . . . .", file=f)

                    variable       =       variables[v]
                    version_string = version_strings[v]
                    
                    

                    mid_thredds_file     = "LOCA2-CONUS___" + variable + "___" + model + "." + member + "___" + scenario + "___" + single_year_string + ".nc"
                    mid_thredds_loc      = thredds_dir + "/" + mid_thredds_file

                    globus_dir  = globus_root_dir + "/" + scenario + "/" + variable                 

                    globus_file      = variable + "." + model + "." + scenario + "." + member + "." + year_string + "." + version_string + ".nc"

                    globus_file_loc  = globus_dir + "/" + globus_file
                    
                    # test
                    
                    if os.path.isfile(globus_file_loc):
                        print("We have our File: ", globus_file_loc)

                        ds = xr.open_dataset(filename_or_obj= globus_file_loc) 

                        if (variable in list(ds.keys())):
                            print("We have our variable: ", list(ds.keys()))


                            print('nohup ncatted -h -O -a           ,global,d,,                 ' + globus_file_loc, file=f)
                            print('nohup ncatted -h -O -a           history,,d,,                 ' + globus_file_loc, file=f)

                            print(" ", file=f)

                            print('nohup ncatted -h -O -a    _FillValue,time,d,,            ' + globus_file_loc, file=f)
                            print('nohup ncatted -h -O -a missing_value,time,d,,            ' + globus_file_loc, file=f)
                            print('nohup ncatted -h -O -a     time_bnds,time,d,,            ' + globus_file_loc, file=f)

                            print(" ", file=f)

                            print('nohup ncatted -h -O -a              ,lat,d,,                 ' + globus_file_loc, file=f)
                            print('nohup ncatted -h -O -a   description,lat,c,c,"Latitude"      ' + globus_file_loc, file=f)
                            print('nohup ncatted -h -O -a     long_name,lat,c,c,"Latitude"      ' + globus_file_loc, file=f)
                            print('nohup ncatted -h -O -a standard_name,lat,c,c,"latitude"      ' + globus_file_loc, file=f)
                            print('nohup ncatted -h -O -a         units,lat,c,c,"degrees_north" ' + globus_file_loc, file=f)
                            print('nohup ncatted -h -O -a          axis,lat,c,c,"Y"             ' + globus_file_loc, file=f)

                            print(" ", file=f)

                            print('nohup ncatted -h -O -a              ,lon,d,,                ' + globus_file_loc, file=f)
                            print('nohup ncatted -h -O -a   description,lon,c,c,"Longitude"    ' + globus_file_loc, file=f)
                            print('nohup ncatted -h -O -a     long_name,lon,c,c,"Longitude"    ' + globus_file_loc, file=f)
                            print('nohup ncatted -h -O -a standard_name,lon,c,c,"longitude"    ' + globus_file_loc, file=f)
                            print('nohup ncatted -h -O -a         units,lon,c,c,"degrees_east" ' + globus_file_loc, file=f)
                            print('nohup ncatted -h -O -a          axis,lon,c,c,"X"            ' + globus_file_loc, file=f)

                            print(" ", file=f)  

                            print('nohup ncks -O -4 -h -d time,' + str(year0) + ',' + str(yearN) + ' ' + globus_file_loc + ' ' + mid_thredds_loc, file=f)

                            print(" ", file=f)

                            print('nohup ' + "ncap2 -O -4 -h -s 'time = double(time);lat = float(lat);lon = float(lon);' " + mid_thredds_loc + ' ' + mid_thredds_loc + '.temp.nc', file=f)
                            print('nohup mv -fv ' + mid_thredds_loc + '.temp.nc' + ' ' +  mid_thredds_loc   , file=f)

                            print(" ", file=f)




                            print(" ", file=f)

                            if (variable == "tasmax"):

                                print(" ", file=f)                  
                                #     'where(Tair &lt; 273.25) snow=prcp; elsewhere rain=prcp;'
                                compress_cmd = "where(tasmax > 1e9) tasmax=short(round(-32767)); elsewhere tasmax=short(round((tasmax-273.15)*10));"

                                print('nohup ncatted -h -O -a              ,tasmax,d,,                                 ' + mid_thredds_loc, file=f)

                                print('nohup ' + "ncap2 -O -4 -h --script '" + compress_cmd + "' " + mid_thredds_loc + ' ' + mid_thredds_loc + '.temp.nc' , file=f)
                                print('nohup mv -fv ' + mid_thredds_loc + '.temp.nc' + ' ' +  mid_thredds_loc   , file=f)

                                print('nohup ' + "ncap2 -O -4 -h --script 'tasmax=short(tasmax)' " + mid_thredds_loc + ' ' + mid_thredds_loc + '.temp.nc' , file=f)
                                print('nohup mv -fv ' + mid_thredds_loc + '.temp.nc' + ' ' +  mid_thredds_loc   , file=f)

                                print(" ", file=f)

                                print('nohup cdo --no_history mul '+  mid_thredds_loc + ' ~/GitHub/LOCA2_Prep/loca2_mask.nc ' + mid_thredds_loc + '.temp.nc', file=f)
                                print('nohup mv -fv '  + mid_thredds_loc + '.temp.nc' + ' ' +  mid_thredds_loc   , file=f)

                                print(" ", file=f)

                                print('nohup ncatted -h -O -a   description,tasmax,c,c,"2-m Max Daily Air Temperature" ' + mid_thredds_loc, file=f)
                                print('nohup ncatted -h -O -a     long_name,tasmax,c,c,"2-m Max Daily Air Temperature" ' + mid_thredds_loc, file=f)
                                print('nohup ncatted -h -O -a standard_name,tasmax,c,c,"air_temperature"               ' + mid_thredds_loc, file=f)
                                print('nohup ncatted -h -O -a         units,tasmax,c,c,"degC"                          ' + mid_thredds_loc, file=f)
                                print('nohup ncatted -h -O -a  cell_methods,tasmax,c,c,"time: maximum"                 ' + mid_thredds_loc, file=f)
                                print('nohup ncatted -h -O -a    _FillValue,tasmax,c,s,-32767                          ' + mid_thredds_loc, file=f)
                                print('nohup ncatted -h -O -a missing_value,tasmax,c,s,-32767                      ' + mid_thredds_loc, file=f)
                                print('nohup ncatted -h -O -a    add_offset,tasmax,c,f,0                               ' + mid_thredds_loc, file=f)
                                print('nohup ncatted -h -O -a  scale_factor,tasmax,c,f,0.1                             ' + mid_thredds_loc, file=f)

                                print(" ", file=f)

                            elif (variable == "tasmin"):


                                compress_cmd = "where(tasmin > 1e9) tasmin=short(round(-32767)); elsewhere tasmin=short(round((tasmin-273.15)*10));"

                                print('nohup ncatted -h -O -a              ,tasmin,d,,                                 ' + mid_thredds_loc, file=f)


                                print('nohup ' + "ncap2 -O -4 -h --script '" + compress_cmd + "' " + mid_thredds_loc + ' ' + mid_thredds_loc + '.temp.nc' , file=f)
                                print('nohup mv -fv ' + mid_thredds_loc + '.temp.nc' + ' ' +  mid_thredds_loc   , file=f)

                                print('nohup ' + "ncap2 -O -4 -h --script 'tasmin=short(tasmin)' " + mid_thredds_loc + ' ' + mid_thredds_loc + '.temp.nc' , file=f)
                                print('nohup mv -fv ' + mid_thredds_loc + '.temp.nc' + ' ' +  mid_thredds_loc   , file=f)


                                print(" ", file=f)

                                print('nohup cdo --no_history mul '+  mid_thredds_loc + ' ~/GitHub/LOCA2_Prep/loca2_mask.nc ' + mid_thredds_loc + '.temp.nc', file=f)
                                print('nohup mv -fv '  + mid_thredds_loc + '.temp.nc' + ' ' +  mid_thredds_loc   , file=f)

                                print(" ", file=f)

                                print('nohup ncatted -h -O -a   description,tasmin,c,c,"2-m Min Daily Air Temperature" ' + mid_thredds_loc, file=f)
                                print('nohup ncatted -h -O -a     long_name,tasmin,c,c,"2-m Min Daily Air Temperature" ' + mid_thredds_loc, file=f)
                                print('nohup ncatted -h -O -a standard_name,tasmin,c,c,"air_temperature"               ' + mid_thredds_loc, file=f)
                                print('nohup ncatted -h -O -a         units,tasmin,c,c,"degC"                          ' + mid_thredds_loc, file=f)
                                print('nohup ncatted -h -O -a  cell_methods,tasmin,c,c,"time: minimum"                 ' + mid_thredds_loc, file=f)
                                print('nohup ncatted -h -O -a    _FillValue,tasmin,c,s,-32767                          ' + mid_thredds_loc, file=f)
                                print('nohup ncatted -h -O -a missing_value,tasmin,c,s,-32767                      ' + mid_thredds_loc, file=f)
                                print('nohup ncatted -h -O -a    add_offset,tasmin,c,f,0                               ' + mid_thredds_loc, file=f)
                                print('nohup ncatted -h -O -a  scale_factor,tasmin,c,f,0.1                             ' + mid_thredds_loc, file=f)

                                print(" ", file=f)

                            else: 


                                compress_cmd = "where(pr > 1e9) pr=short(round(-32767)); elsewhere pr=short(round(pr*864000.));"

                                print('nohup ncatted -h -O -a              ,pr,d,,                             ' + mid_thredds_loc, file=f)

                                print("nohup ncap2 -O -4 -h --script '" + compress_cmd + "' " + mid_thredds_loc + ' ' + mid_thredds_loc + '.temp.nc' , file=f)
                                print('nohup mv -fv ' + mid_thredds_loc + '.temp.nc' + ' ' +  mid_thredds_loc   , file=f)
                                print('nohup ' + "ncap2 -O -4 -h --script 'pr=short(pr)' " + mid_thredds_loc + ' ' + mid_thredds_loc + '.temp.nc' , file=f)
                                print('nohup mv -fv ' + mid_thredds_loc + '.temp.nc' + ' ' +  mid_thredds_loc   , file=f)

                                print(" ", file=f)

                                print('nohup cdo --no_history mul '+  mid_thredds_loc + ' ~/GitHub/LOCA2_Prep/loca2_mask.nc ' + mid_thredds_loc + '.temp.nc', file=f)
                                print('nohup mv -fv '  + mid_thredds_loc + '.temp.nc' + ' ' +  mid_thredds_loc   , file=f)

                                print(" ", file=f)

                                print('nohup ncatted -h -O -a   description,pr,c,c,"Daily Total Precipitation" ' + mid_thredds_loc, file=f)
                                print('nohup ncatted -h -O -a     long_name,pr,c,c,"Daily Total Precipitation" ' + mid_thredds_loc, file=f)
                                print('nohup ncatted -h -O -a standard_name,pr,c,c,"precipitation_amount"      ' + mid_thredds_loc, file=f)
                                print('nohup ncatted -h -O -a         units,pr,c,c,"kg m-2"                    ' + mid_thredds_loc, file=f)
                                print('nohup ncatted -h -O -a  cell_methods,pr,c,c,"time: sum"                 ' + mid_thredds_loc, file=f)
                                print('nohup ncatted -h -O -a    _FillValue,pr,c,s,-32767                      ' + mid_thredds_loc, file=f)
                                print('nohup ncatted -h -O -a missing_value,pr,c,s,-32767                      ' + mid_thredds_loc, file=f)
                                print('nohup ncatted -h -O -a    add_offset,pr,c,f,0                           ' + mid_thredds_loc, file=f)
                                print('nohup ncatted -h -O -a  scale_factor,pr,c,f,0.1                         ' + mid_thredds_loc, file=f)

                                print(" ", file=f)

                            print(" ", file=f)
                            print("# . . . . . . . . . . . . . . . . . . . . .", file=f)
                    
                print("# .........................................", file=f)
                year0 = yearN+1
                print('nohup rm -frv  '+ cat_thredds_loc, file=f)
                print('nohup cdo  --no_history -f nc4 merge '+ mid_wildthredds_loc + ' ' + cat_thredds_loc, file=f)
                print(" ", file=f)
                print('nohup ' + "ncap2 -O -4 -h -s '" + bounds_script_command + "' " + cat_thredds_loc + ' ' + cat_thredds_loc + '.temp.nc', file=f)
                print('nohup mv -fv ' + cat_thredds_loc + '.temp.nc' + ' ' +  cat_thredds_loc   , file=f)
                print(" ", file=f)
                print('rm  '+ mid_wildthredds_loc, file=f)

                print('nohup ncatted -h -O -a  title,global,c,c,"LOCA statistically downscaled climate model data, David W. Pierce, Scripps Institution of Oceanography" ' + cat_thredds_loc, file=f)
                print('nohup ncatted -h -O -a  author,global,c,c,"David W. Pierce, Scripps Institution of Oceanography" ' + cat_thredds_loc, file=f)
                print('nohup ncatted -h -O -a  cmip_id,global,c,c,"CMIP6" '                         + cat_thredds_loc, file=f)
                print('nohup ncatted -h -O -a  experiment_id,global,c,c,"'    + scenario + '" '     + cat_thredds_loc, file=f)
                print('nohup ncatted -h -O -a  model_source_id,global,c,c,"'  + model    + '" '     + cat_thredds_loc, file=f)
                print('nohup ncatted -h -O -a  model_variant_id,global,c,c,"' + member   + '" '     + cat_thredds_loc, file=f)
                print('nohup ncatted -h -O -a  Conventions,global,c,c,"CF-1.7 CMIP-6.0 UGRID-1.0" ' + cat_thredds_loc, file=f)
                
                print("# -----------------------------------------", file=f)
            
                
            print("# =========================================", file=f)   
        print(" ", file=f)
        print("nohup rm -frv  " +  fnl_thredds_loc, file=f)
        print("nohup cdo  --no_history -f nc4 -z zip_8  mergetime " + cat_thredds_all_yrs_loc + " " + fnl_thredds_loc, file=f)
        print('rm ' + cat_thredds_all_yrs_loc  , file=f)
            
        print("# XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX", file=f)

        f.close()
        os.system("chmod a+x " + script_name)            

We have our File:  /data/GLOBUS/wjc/historical/tasmax/tasmax.IPSL-CM6A-LR.historical.r1i1p1f1.1950-2014.LOCA_16thdeg_v20220413.nc
We have our variable:  ['tasmax']
We have our File:  /data/GLOBUS/wjc/historical/tasmin/tasmin.IPSL-CM6A-LR.historical.r1i1p1f1.1950-2014.LOCA_16thdeg_v20220413.nc
We have our variable:  ['tasmin']
We have our File:  /data/GLOBUS/wjc/historical/pr/pr.IPSL-CM6A-LR.historical.r1i1p1f1.1950-2014.LOCA_16thdeg_v20220519.nc
We have our variable:  ['pr']
We have our File:  /data/GLOBUS/wjc/historical/tasmax/tasmax.IPSL-CM6A-LR.historical.r1i1p1f1.1950-2014.LOCA_16thdeg_v20220413.nc
We have our variable:  ['tasmax']
We have our File:  /data/GLOBUS/wjc/historical/tasmin/tasmin.IPSL-CM6A-LR.historical.r1i1p1f1.1950-2014.LOCA_16thdeg_v20220413.nc
We have our variable:  ['tasmin']
We have our File:  /data/GLOBUS/wjc/historical/pr/pr.IPSL-CM6A-LR.historical.r1i1p1f1.1950-2014.LOCA_16thdeg_v20220519.nc
We have our variable:  ['pr']
We have our File:  /data/GLOBUS/wjc/hist

In [9]:
print("end!")

end!
